In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data2 = pd.read_csv("C:/Users/zhang/Desktop/NYU/2019 Fall/Humane/Data/HMAHCC_COMP.csv")

D:\Downloads\Anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [245]:
data = data2[data2.index <= 50000]

In [246]:
data.drop_duplicates(inplace = True)

D:\Downloads\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [247]:
# Find <id, Days> where 'RX Claim - Paid' or 'RX Claim - New Drug' or 'RX CLaim - First Time Main Order' has the same generic as
# 'RX Claim - Rejected' when 'RX Claim - Rejected' == 'REVERSAL'

# We only care about Opioid rows
sub_data = data[(data['event_attr1'] == 'REVERSAL') | (data['MME'].isna() == False)]
sub_data['Generic'] = np.where(sub_data['event_descr'].isin(['RX Claim - Rejected']), \
                               sub_data['event_attr6'], sub_data['event_attr8'])

D:\Downloads\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [248]:
# Only include Opioid generic

opioid_generic = sub_data[sub_data['event_descr'] == 'RX Claim - Paid']['Generic'].unique()
sub_data = sub_data[sub_data['Generic'].isin(opioid_generic)]

In [249]:
# Running on Colab

id_day = []
for uid in sub_data['id'].unique():
    for day in sub_data[sub_data['id'] == uid]['Days'].unique():
        sample = sub_data[(sub_data['id'] == uid) & (sub_data['Days'] == day)]
        if 'REVERSAL' in sample['event_attr1'].unique() and sample['event_descr'].nunique() >= 2\
        and sample['Generic'].nunique() < sample.shape[0] :
            id_day.append([uid, day])

In [250]:
id_day

[['ID10230809175', 800], ['ID10522725996', 470], ['ID10680847713', 707]]

In [251]:
# ????? need more detail to be explained

for uid, day in id_day:
    for generic in data[(data['id'] == uid) & (data['Days'] == day) & (data['event_attr1'] == 'REVERSAL')]['event_attr6'].unique():
        if data[(data['id'] == uid) & (data['Days'] == day) & \
                (data['event_descr'] == 'RX Claim - Paid') & (data['event_attr8'] == generic)].shape[0] > 0:
            
            row_index_paid = data[(data['id'] == uid) & (data['Days'] == day) & \
                                  (data['event_descr'] == 'RX Claim - Paid') & (data['event_attr8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_paid, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_paid, 'PAYABLE_QTY'] = 0
                data.at[row_index_paid, 'MME'] = 0
            
        if data[(data['id'] == uid) & (data['Days'] == day) & \
                (data['event_descr'] == 'RX Claim - New Drug') & (data['event_attr8'] == generic)].shape[0] > 0:
            
            row_index_new = data[(data['id'] == uid) & (data['Days'] == day) & \
                                  (data['event_descr'] == 'RX Claim - New Drug') & (data['event_attr8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_new, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_new, 'PAYABLE_QTY'] = 0
                data.at[row_index_new, 'MME'] = 0
            
        if data[(data['id'] == uid) & (data['Days'] == day) & \
                (data['event_descr'] == 'RX Claim - First Time Mail Order') & (data['event_attr8'] == generic)].shape[0] > 0:
            
            row_index_mail = data[(data['id'] == uid) & (data['Days'] == day) & \
                                 (data['event_descr'] == 'RX Claim - First Time Mail Order') & \
                                 (data['event_attr8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_mail, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_mail, 'PAYABLE_QTY'] = 0
                data.at[row_index_mail, 'MME'] = 0

D:\Downloads\Anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [254]:
# Find duplicates of "RX Claim Paid", "RX Claim New Drug", 'RX Claim First Time Mail Order' in the same day, if opioid
# Pick the max supply day and change the other as 0

opioid_data = data[data['MME'].isna() == False]
opioid_data = opioid_data[opioid_data.duplicated(['id', 'event_descr', 'Days'], keep = False)]
opioid_data_max_supply = opioid_data.sort_values(['id', 'event_descr', 'Days', 'PAY_DAY_SUPPLY_CNT'])\
                                    .drop_duplicates(['id', 'event_descr', 'Days'], keep = 'last')
change_id = np.setdiff1d(np.array(opioid_data.index), np.array(opioid_data_max_supply.index))
for uid in c`hange_id:
    data.loc[uid, 'PAY_DAY_SUPPLY_CNT'] = 0

D:\Downloads\Anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [312]:
feature = data[data['Days'] < 0]
target = data[data['Days'] >= 0]

In [256]:
# Extract total other diagnosis times, total cost, total paid
# What is not extracted: drug class, brand name, drug description
def RX_claim_paid_other(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - Paid') & (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_paid_other = instance.shape[0]
        total_RX_claim_paid_cost_other = instance['event_attr3'].astype('float32').sum()
        total_RX_claim_paid_paid_other = instance['event_attr4'].astype('float32').sum()
    if instance.shape[0] == 0:    
        total_RX_claim_paid_other = 0
        total_RX_claim_paid_cost_other = 0
        total_RX_claim_paid_paid_other = 0   
        
    return [total_RX_claim_paid_other, total_RX_claim_paid_cost_other, total_RX_claim_paid_paid_other]

In [257]:
# Extract total opioid times, total cost, total paid, supply count, quantity, MME
# What is not extracted: brand name, drug description, drug type, Specialty, Speciatly2, Specialty3
def RX_claim_paid_opioid(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - Paid') & (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_paid_opioid = instance.shape[0]
        total_RX_claim_paid_cost_opioid = instance['event_attr3'].astype('float32').sum()
        total_RX_claim_paid_paid_opioid = instance['event_attr4'].astype('float32').sum()
        total_RX_claim_paid_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_paid_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_paid_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_paid_opioid = 0
        total_RX_claim_paid_cost_opioid = 0
        total_RX_claim_paid_paid_opioid = 0
        total_RX_claim_paid_supply_opioid = 0
        total_RX_claim_paid_quantity_opioid = 0
        total_RX_claim_paid_MME_opioid = 0       
        
    return [total_RX_claim_paid_opioid, total_RX_claim_paid_cost_opioid, total_RX_claim_paid_paid_opioid,\
            total_RX_claim_paid_supply_opioid, total_RX_claim_paid_quantity_opioid, total_RX_claim_paid_MME_opioid]

In [258]:
# Extract total other new drug times

def RX_claim_new_other(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - New Drug') &\
                       (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_new_other = instance.shape[0]
    if instance.shape[0] == 0:    
        total_RX_claim_new_other = 0  
        
    return [total_RX_claim_new_other]

In [259]:
# Extract new drug opioid times, total cost, total paid, supply count, quantity, MME

def RX_claim_new_opioid(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - New Drug') &\
                       (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_new_opioid = instance.shape[0]
        total_RX_claim_new_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_new_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_new_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_new_opioid = 0
        total_RX_claim_new_supply_opioid = 0
        total_RX_claim_new_quantity_opioid = 0
        total_RX_claim_new_MME_opioid = 0       
        
    return [total_RX_claim_new_opioid, total_RX_claim_new_supply_opioid, total_RX_claim_new_quantity_opioid,\
            total_RX_claim_new_MME_opioid]

In [260]:
# Extract total other first mail times

def RX_claim_mail_other(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - First Time Mail Order') &\
                       (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_mail_other = instance.shape[0]
    if instance.shape[0] == 0:    
        total_RX_claim_mail_other = 0  
        
    return [total_RX_claim_mail_other]

In [261]:
# Extract opioid first mail times, supply count, quantity, MME

def RX_claim_mail_opioid(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - First Time Mail Order') &\
                       (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_mail_opioid = instance.shape[0]
        total_RX_claim_mail_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_mail_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_mail_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_mail_opioid = 0
        total_RX_claim_mail_supply_opioid = 0
        total_RX_claim_mail_quantity_opioid = 0
        total_RX_claim_mail_MME_opioid = 0       
        
    return [total_RX_claim_mail_opioid, total_RX_claim_mail_supply_opioid, total_RX_claim_mail_quantity_opioid,\
            total_RX_claim_mail_MME_opioid]

In [262]:
# Extract total reject times

def RX_claim_reject(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'RX Claim - Reject')]
    if instance.shape[0] != 0:
        total_RX_claim_reject = instance.shape[0]
    if instance.shape[0] == 0:    
        total_RX_claim_reject = 0  
        
    return [total_RX_claim_reject]

In [263]:
# Extract total inbound call by mbr times

def Inbound_call_by_mbr(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'Inbound Call by Mbr')]
    if instance.shape[0] != 0:
        inbound_call_by_mbr = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_mbr = 0  
        
    return [inbound_call_by_mbr]

In [264]:
# Extract total inbound call by other times

def Inbound_call_by_other(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'Inbound Call by Other')]
    if instance.shape[0] != 0:
        inbound_call_by_other = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_other = 0  
        
    return [inbound_call_by_other]

In [265]:
# Extract total inbound call by prov times

def Inbound_call_by_prov(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'Inbound Call by Prov')]
    if instance.shape[0] != 0:
        inbound_call_by_prov = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_prov = 0  
        
    return [inbound_call_by_prov]

In [266]:
# Extract total fully paid claim times, total charge, total net paid, total responsible amount

def Fully_paid_claim(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'Fully Paid Claim')]
    if instance.shape[0] != 0:
        fully_paid_claim = instance.shape[0]
        fully_paid_claim_charge = instance['event_attr3'].astype('float32').sum()
        fully_paid_claim_paid = instance['event_attr4'].astype('float32').sum()
        fully_paid_claim_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        fully_paid_claim = 0
        fully_paid_claim_charge = 0
        fully_paid_claim_paid = 0
        fully_paid_claim_responsible = 0   
        
    return [fully_paid_claim, fully_paid_claim_charge, fully_paid_claim_paid, fully_paid_claim_responsible]

In [267]:
# Extract total New Diagonosis Top 5 claim times, total charge, total net paid, total responsible amount

def Top_five(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - Top 5')]
    if instance.shape[0] != 0:
        top_five = instance.shape[0]
        top_five_charge = instance['event_attr3'].astype('float32').sum()
        top_five_paid = instance['event_attr4'].astype('float32').sum()
        top_five_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        top_five = 0
        top_five_charge = 0
        top_five_paid = 0
        top_five_responsible = 0   
        
    return [top_five, top_five_charge, top_five_paid, top_five_responsible]

In [268]:
# Extract total New Diagonosis CAD claim times, total charge, total net paid, total responsible amount

def CAD(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - CAD')]
    if instance.shape[0] != 0:
        cad = instance.shape[0]
        cad_charge = instance['event_attr3'].astype('float32').sum()
        cad_paid = instance['event_attr4'].astype('float32').sum()
        cad_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        cad = 0
        cad_charge = 0
        cad_paid = 0
        cad_responsible = 0   
        
    return [cad, cad_charge, cad_paid, cad_responsible]

In [269]:
# Extract total New Diagonosis Diabetes claim times, total charge, total net paid, total responsible amount

def Diabetes(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - Diabetes')]
    if instance.shape[0] != 0:
        diabetes = instance.shape[0]
        diabetes_charge = instance['event_attr3'].astype('float32').sum()
        diabetes_paid = instance['event_attr4'].astype('float32').sum()
        diabetes_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        diabetes = 0
        diabetes_charge = 0
        diabetes_paid = 0
        diabetes_responsible = 0   
        
    return [diabetes, diabetes_charge, diabetes_paid, diabetes_responsible]

In [270]:
# Extract total New Diagonosis Hypertension claim times, total charge, total net paid, total responsible amount

def Hypertension(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - Hypertension')]
    if instance.shape[0] != 0:
        hypertension = instance.shape[0]
        hypertension_charge = instance['event_attr3'].astype('float32').sum()
        hypertension_paid = instance['event_attr4'].astype('float32').sum()
        hypertension_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        hypertension = 0
        hypertension_charge = 0
        hypertension_paid = 0
        hypertension_responsible = 0   
        
    return [hypertension, hypertension_charge, hypertension_paid, hypertension_responsible]

In [271]:
# Extract total New Diagonosis CPD claim times, total charge, total net paid, total responsible amount

def CPD(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - CPD')]
    if instance.shape[0] != 0:
        cpd = instance.shape[0]
        cpd_charge = instance['event_attr3'].astype('float32').sum()
        cpd_paid = instance['event_attr4'].astype('float32').sum()
        cpd_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        cpd = 0
        cpd_charge = 0
        cpd_paid = 0
        cpd_responsible = 0   
        
    return [cpd, cpd_charge, cpd_paid, cpd_responsible]

In [272]:
# Extract total New Diagonosis CHF claim times, total charge, total net paid, total responsible amount

def CHF(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'New diagnosis - CHF')]
    if instance.shape[0] != 0:
        chf = instance.shape[0]
        chf_charge = instance['event_attr3'].astype('float32').sum()
        chf_paid = instance['event_attr4'].astype('float32').sum()
        chf_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        chf = 0
        chf_charge = 0
        chf_paid = 0
        chf_responsible = 0   
        
    return [chf, chf_charge, chf_paid, chf_responsible]

In [273]:
# Extract total Surgery claim times, total charge, total net paid, total responsible amount

def Surgery(uid):
    instance = feature[(feature['id'] == uid) & (feature['event_descr'] == 'Surgery')]
    if instance.shape[0] != 0:
        surgery = instance.shape[0]
        surgery_charge = instance['event_attr3'].astype('float32').sum()
        surgery_paid = instance['event_attr4'].astype('float32').sum()
        surgery_responsible = instance['event_attr5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        surgery = 0
        surgery_charge = 0
        surgery_paid = 0
        surgery_responsible = 0   
        
    return [surgery, surgery_charge, surgery_paid, surgery_responsible]

In [274]:
columns = ['id', 'total_RX_claim_paid_other', 'total_RX_claim_paid_cost_other', 'total_RX_claim_paid_paid_other', \
           'total_RX_claim_paid_opioid', 'total_RX_claim_paid_cost_opioid', 'total_RX_claim_paid_paid_opioid', \
           'total_RX_claim_paid_supply_opioid', 'total_RX_claim_paid_quantity_opioid', 'total_RX_claim_paid_MME_opioid', \
           'total_RX_claim_new_other', 'total_RX_claim_new_opioid', 'total_RX_claim_new_supply_opioid', \
           'total_RX_claim_new_quantity_opioid', 'total_RX_claim_new_MME_opioid', 'total_RX_claim_mail_other', \
           'total_RX_claim_mail_opioid', 'total_RX_claim_mail_supply_opioid', 'total_RX_claim_mail_quantity_opioid', \
           'total_RX_claim_mail_MME_opioid', 'total_RX_claim_reject', 'inbound_call_by_mbr', 'inbound_call_by_other', \
           'inbound_call_by_prov', 'fully_paid_claim', 'fully_paid_claim_charge', 'fully_paid_claim_paid', \
           'fully_paid_claim_responsible', 'top_five', 'top_five_charge', 'top_five_paid', 'top_five_responsible', 'cad', \
           'cad_charge', 'cad_paid', 'cad_responsible', 'diabetes', 'diabetes_charge', 'diabetes_paid', 'diabetes_responsible', \
           'hypertension', 'hypertension_charge', 'hypertension_paid', 'hypertension_responsible', 'cpd', 'cpd_charge', \
           'cpd_paid', 'cpd_responsible', 'chf', 'chf_charge', 'chf_paid', 'chf_responsible', 'surgery', 'surgery_charge', \
           'surgery_paid', 'surgery_responsible']

In [296]:
processed_feature = pd.DataFrame(columns = columns)

In [297]:
for uid in feature['id'].unique():
    one_row = []
    one_row.append(uid)
    one_row.extend(RX_claim_paid_other(uid))
    one_row.extend(RX_claim_paid_opioid(uid))
    one_row.extend(RX_claim_new_other(uid))
    one_row.extend(RX_claim_new_opioid(uid))
    one_row.extend(RX_claim_mail_other(uid))
    one_row.extend(RX_claim_mail_opioid(uid))
    one_row.extend(RX_claim_reject(uid))
    one_row.extend(Inbound_call_by_mbr(uid))
    one_row.extend(Inbound_call_by_other(uid))
    one_row.extend(Inbound_call_by_prov(uid))
    one_row.extend(Fully_paid_claim(uid))
    one_row.extend(Top_five(uid))
    one_row.extend(CAD(uid))
    one_row.extend(Diabetes(uid))
    one_row.extend(Hypertension(uid))
    one_row.extend(CPD(uid))
    one_row.extend(CHF(uid))
    one_row.extend(Surgery(uid))
    processed_feature = processed_feature.append(pd.Series(one_row, index = processed_feature.columns), ignore_index = True)

In [284]:
target_opioid = target[(target['event_descr'] == 'RX Claim - Paid') & (target_drug['MME'].isna() == False)]

In [286]:
ltot_index = []
for uid in target_opioid['id'].unique():
    for day in target_opioid[target_opioid['id'] == uid]['Days'].unique():
        if day == 0:
            ltot = target_opioid[(target_opioid['id'] == uid) & (target_opioid['Days'] <= 179)]
            if ltot['PAY_DAY_SUPPLY_CNT'].sum() >= 162:
                ltot_index.append(uid)
                break
        elif day >= 90:
            opoid_naive = target_opioid[(target_opioid['id'] == uid) & (target_opioid['Days'] <= (day - 1)) &\
                                        (target_opioid['Days'] >= (day - 90))]
            if opoid_naive[opoid_naive['PAY_DAY_SUPPLY_CNT'].isna() == False].shape[0] == 0:
                ltot = target_opioid[(target_opioid['id'] == uid) & (target_opioid['Days'] >= day) &\
                                     (target_opioid['Days'] >= day + 179)]
                if ltot['PAY_DAY_SUPPLY_CNT'].sum() >= 162:
                    ltot_index.append(uid)
                    break

In [293]:
processed_feature['label'] = np.where(processed_feature['id'].isin(ltot_index), 1, 0)